In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import nibabel as nib
import matplotlib.pyplot as plt
# there was some problems for using import tensorflow as tf
# the way I found to fix it is the next two lines
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()

import tensorflow as tf
import tensorflow.keras
import numpy as np
import os

import random

from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Activation, GlobalAveragePooling2D, Dense, GlobalAveragePooling3D
from tensorflow.keras.layers import Input, Conv2D, ZeroPadding2D, BatchNormalization
from tensorflow.keras.layers import Conv3D,MaxPooling3D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.utils import to_categorical
from scipy.ndimage import zoom

from tensorflow.keras.applications.vgg16 import VGG16

#reduce the size of model weights
from tensorflow.keras import backend as K
K.set_floatx('float16')

W1207 22:46:35.583787 140130305517312 deprecation.py:323] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

#maybe I do not need this in future
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        elif fullPath[-4:]=='.nii': # to avoid DS_Store or other system files
            allFiles.append(fullPath)
                
    return allFiles

In [3]:
def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)
    
#     X_conv = Conv3D(150, (5, 5, 5), strides = (1, 1,1), name = 'conv0', data_format="channels_last", activation='sigmoid', trainable=False)(X_input)
    X_conv = Conv3D(30, (5, 5, 5), strides = (1, 1,1), name = 'conv0', data_format="channels_last", activation='sigmoid')(X_input)
    
    X_conv = Conv3D(30, (5, 5, 5), strides = (1, 1,1), name = 'conv0', data_format="channels_last", activation='sigmoid')(X_input)

    X_pool = MaxPooling3D(pool_size=(5, 5, 5), strides=None, data_format=None, name = 'pool')(X_conv) # in the paper it is 5,5,5
    
    X_pool_flat = Flatten()(X_pool)
    
    X_dense = Dense(200, activation='sigmoid', name='fc')(X_pool_flat) #400 is 800 in the paper
    
    X_out = Dense(1, activation='sigmoid', name='out')(X_dense)
    
    model = Model(inputs = X_input, outputs = X_out, name='HappyModel')    
    
    
#     model.load_weights()
    
    return model

In [4]:
model1 = model ((110,110,110,1))

W1207 22:46:47.365651 140130305517312 deprecation.py:506] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
#load model weights, note that we will only use the weights for the convolution layer
model1.load_weights('my_model_weights_1.h5')

In [6]:
model1.summary()

Model: "HappyModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 110, 110, 110, 1) 0         
_________________________________________________________________
conv0 (Conv3D)               (None, 106, 106, 106, 150 18900     
_________________________________________________________________
pool (MaxPooling3D)          (None, 21, 21, 21, 150)   0         
_________________________________________________________________
flatten (Flatten)            (None, 1389150)           0         
_________________________________________________________________
fc (Dense)                   (None, 200)               277830200 
_________________________________________________________________
out (Dense)                  (None, 1)                 201       
Total params: 277,849,301
Trainable params: 277,849,301
Non-trainable params: 0
__________________________________________

In [7]:
#Parameters (Modify as needed)
img_size_x = 110
img_size_y = 110
img_size_z = 110

batch_size = 128
nb_classes = 3
nb_epoch = 25

c = 0

learning_rate = 0.001
early_stopping_patience = 20

class_names = ["CN", "MCI", "AD"]

In [8]:
# # DATA CLEANING

# # Get folder of nii files only - find Documents/CS230/MRI_git/ADNI -
# # type f -print0 | xargs -0 mv -t Documents/CS230/MRI_git/ADNI_Clean/

# # Change names of nii files
# directory = "./../../../../../Documents/Stanford/cs230/ADNI/ADNI_Clean/"
# for filename in os.listdir(directory):
#     if filename.startswith("ADNI"):
#         os.rename(directory+filename, directory+filename[5:16]+".nii")

In [9]:
def getImage( subject, directory ):

    epi_img = nib.load(directory + subject + '.nii')
    epi_img_data = epi_img.get_fdata()
        
    return epi_img_data

In [10]:
# def load_dataset(directory):
    
#     patientData = np.loadtxt("ADNI1_Complete_1Yr_1.5T_10_26_2019.csv", dtype= 'str', skiprows=1, delimiter=',')
    
#     integer_mapping = {x: i for i,x in enumerate(['AD', 'MCI', 'CN'])}
#     y = np.asarray([integer_mapping[word] for word in patientData[:,1]])
#     labels = to_categorical(y, num_classes=3, dtype='float32')
    
    
    
#     xdim = 68
#     ydim = 95
#     zdim = 79
#     X = np.zeros((637,xdim,ydim,zdim,1))
#     Y = np.zeros((637,3))
#     names = []
#     for i, filename in enumerate(os.listdir(directory)):
#         if i == 150:
#             break
#         if filename[-4:]!='.nii':
#             continue
        
#         subject = filename[:-4]
        
#         # Get voxel array
#         epi_img_data = getImage( subject, directory )
        
#         n_i, n_j, n_k = epi_img_data.shape
        
#         if (n_i != xdim or n_j != ydim or n_k != zdim):
#             #epi_img_data = cv2.resize(epi_img_data, (192, 192))
#             #epi_img_data = epi_img_data[0:192,0:192,int(n_k/2)-80:int(n_k/2)+80]
            
#             #resampling to make all MRI volumes the same dimensions
#             epi_img_data = zoom(epi_img_data, (float(xdim/n_i), float(ydim/n_j), float(zdim/n_k)), order = 0)
        
#         x = epi_img_data
        
#         x = np.expand_dims(x, axis=3)
        
#         X[i] = x
#         ind = np.where(patientData[:,0] == subject[:-1])[0][0]
#         y = labels[ind, :]
#         y = y.reshape(1,-1)
#         names.append(subject)
        
#         Y[i] = y
        
#         if i % 50 == 0:
#             print ("loaded " + str(i) + "subjects")
        
        
#     print (X.shape)
#     print (Y.shape)
        
#     return X, Y, names

In [11]:
def load_dataset_clean():
    AD_directory = "/home/ubuntu/project/Data/AD_clean/"
    NC_directory = "/home/ubuntu/project/Data/NC_clean/"
    
    AD_list = os.listdir(AD_directory)
    NC_list = os.listdir(NC_directory)
    

    All_directory = "/home/ubuntu/project/Data/AD_NC_Clean/"

    X = np.zeros((111,img_size_x,img_size_y,img_size_z,1))
    Y = np.zeros(111)
    
    l = os.listdir(All_directory)
    random.shuffle(l)
   
    
    objindex = 0
    for i, filename in enumerate(l):
        if (filename.startswith('.')):
            print ("hidden file")
        else:
            epi_img = nib.load(All_directory + filename)
            x = epi_img.get_fdata()
            x = (x-x.min())/x.max()
            
            x = np.expand_dims(x, axis=3)
            X[objindex] = x
            y = 0
            if filename in AD_list:
                y = 1
                
            Y[objindex] = y
            
            objindex += 1
            
    return X, Y

In [12]:
X, Y = load_dataset_clean()

In [13]:
print(Y.shape)
print(X.shape)

(111,)
(111, 110, 110, 110, 1)


In [16]:
#load model weights, note that we will only use the weights for the convolution layer
from tensorflow.keras import losses
# model1.load_weights('my_model_weights_0_14.h5')
# opt = keras.optimizers.Adadelta(clipnorm=1.)
opt = tf.keras.optimizers.Adam(lr=0.00005,beta_1=0.9, beta_2=0.99, amsgrad=False)
# def batch_callback():
#     print('hello')
    
model1.compile(optimizer=opt,loss='binary_crossentropy', metrics=['accuracy'])

# model1.compile(loss=losses.mean_squared_error, optimizer='sgd', metrics=['categorical_accuracy'])


m = model1.fit(X[0:70],Y[0:70],validation_data=(X[70:100],Y[70:100]),batch_size=2,epochs=20, verbose=1)
# model1.train_on_batch(X[0:10], Y[0:10])

Train on 70 samples, validate on 30 samples
Epoch 1/20
70/70 [==============================] - 25s 359ms/sample - loss: 0.7073 - acc: 0.4570 - val_loss: 0.7169 - val_acc: 0.4666
Epoch 2/20
70/70 [==============================] - 25s 352ms/sample - loss: 0.7019 - acc: 0.4570 - val_loss: 0.7132 - val_acc: 0.4666
Epoch 3/20
70/70 [==============================] - 25s 353ms/sample - loss: 0.6988 - acc: 0.4570 - val_loss: 0.7120 - val_acc: 0.4666
Epoch 4/20
70/70 [==============================] - 25s 354ms/sample - loss: 0.6967 - acc: 0.4570 - val_loss: 0.7085 - val_acc: 0.4666
Epoch 5/20
70/70 [==============================] - 25s 356ms/sample - loss: 0.6947 - acc: 0.4714 - val_loss: 0.7081 - val_acc: 0.4666
Epoch 6/20
70/70 [==============================] - 25s 354ms/sample - loss: 0.6940 - acc: 0.4570 - val_loss: 0.7062 - val_acc: 0.4666
Epoch 7/20
70/70 [==============================] - 25s 354ms/sample - loss: 0.6918 - acc: 0.4570 - val_loss: 0.7060 - val_acc: 0.4666
Epoch 8/20


In [33]:
model1.save_weights('my_model_weights_0_1398.h5')

In [16]:
classes = model1.predict(X[0:10])
# -np.sum(np.log(classes)*Y[0:10])/10
print(classes)

[[1.466e-05 9.971e-01 2.689e-03]
 [5.327e-01 4.663e-01 9.298e-04]
 [2.548e-02 2.588e-02 9.487e-01]
 [1.000e+00 3.576e-07 4.053e-06]
 [5.327e-01 4.663e-01 9.298e-04]
 [9.058e-01 9.436e-02 6.974e-06]
 [5.264e-01 4.729e-01 8.540e-04]
 [4.768e-07 9.995e-01 4.499e-04]
 [2.548e-02 2.588e-02 9.487e-01]
 [1.478e-05 9.971e-01 2.701e-03]]


In [17]:
print(Y[0:10])

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [18]:
x_test = X[0:10]
y_test = Y[0:10]
loss_and_metrics = model1.evaluate(x_test, y_test, batch_size=128)

10/10 [==============================] - 0s 46ms/sample - loss: 5.0312 - categorical_accuracy: 0.2000


In [38]:
print(type(loss_and_metrics[0]))

<class 'numpy.float64'>


In [18]:
print(loss_and_metrics[0])

0.96337890625


In [19]:
print(loss_and_metrics[1])

0.5


In [20]:
classes = model1.predict(X[0:10])
# -np.sum(np.log(classes)*Y[0:10])/10
print(classes)

[[0.2344 0.4507 0.315 ]
 [0.2335 0.4526 0.314 ]
 [0.2129 0.4749 0.3123]
 [0.2134 0.4727 0.314 ]
 [0.2313 0.451  0.3179]
 [0.2242 0.4683 0.3074]
 [0.4214 0.343  0.2357]
 [0.2327 0.452  0.3154]
 [0.2133 0.4746 0.3123]
 [0.2131 0.474  0.3127]]


In [65]:
print(np.sum(classes,axis=0))

[0.1117 9.86   0.0399]


In [14]:
print(np.sum(Y[90:100],axis=0))

[3. 5. 2.]


In [65]:
m = model1.fit(X[100:200],Y[100:200],batch_size=20,epochs=3, verbose=2)

Epoch 1/3
 - 4s - loss: 1.0896 - categorical_accuracy: 0.3899
Epoch 2/3
 - 3s - loss: 1.0803 - categorical_accuracy: 0.3200
Epoch 3/3
 - 4s - loss: 1.0703 - categorical_accuracy: 0.3700


In [25]:
classes = model1.predict(X[0:10])

In [1]:
classes


NameError: name 'classes' is not defined

In [ ]:
def modelpoly():
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)
    
    X_dense1 = Dense(100, activation='sigmoid', name='fc')(X_input) #400 is 800 in the paper
    
    X_dense1 = Dense(100, activation='sigmoid', name='fc')(X_input)
    
    model = Model(inputs = X_input, outputs = X_out, name='HappyModel')    
    
    
#     model.load_weights()
    
    return model